<a href="https://colab.research.google.com/github/aryanc55/LTFSDSFinHack3/blob/main/Copy_of_FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
!pip install pycaret
!pip install category_encoders

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 81kB 5.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


     |████████████████████████████████| 81kB 5.1MB/s 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from functools import reduce
from datetime import datetime, timedelta
from pycaret.classification import *

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_b = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Train/train_bureau.xlsx')

In [5]:
train_d = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Train/train_Data.xlsx')

In [39]:
train_b.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [8]:
test_d = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Test/test_Data.xlsx')

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.0,365000.0,1000.0,2011-12-29,2015-10-05,2011-12-29,3524747,1046.0,22354,75.83,M,50.0,32069.00,GUNA,MADHYA PRADESH,473001.0
1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,285000.0,285000.0,9300.0,2012-04-28,2016-01-01,2012-04-28,4985862,1046.0,22354,57.44,M,35.0,25000.00,GUNA,MADHYA PRADESH,473001.0
2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,400000.0,400000.0,35800.0,2013-10-22,2017-09-10,2013-10-22,12881783,1060.0,65929,68.97,M,37.0,23333.33,GUNA,MADHYA PRADESH,473001.0
3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,500000.0,500000.0,52000.0,2013-05-21,2016-02-10,2013-05-21,10613776,1049.0,68597,61.79,M,55.0,91666.67,MAHASAMUND,CHATTISGARH,493558.0
4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,400000.0,400000.0,35400.0,2012-11-30,2016-12-05,2012-11-30,7911110,1049.0,38493,59.09,M,48.0,12500.00,JAGATSINGHAPUR,ORISSA,754137.0


In [46]:
test_b = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Test/test_bureau.xlsx')

In [47]:
test_b.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,NaN,589,NaN,0,0.0,NaN,"20191117,",000,"589,","0,","0,",NaN
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,"25,00,000","30,797/Monthly","21,07,227",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,"99,000","1,223/Monthly","83,458",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,"3,65,000",NaN,0,NaN,0,0.0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,"25,801",NaN,0,NaN,0,0.0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",NaN


In [52]:
train_bb = train_b.copy()

In [53]:
test_bb = test_b.copy()

In [12]:
# Data Treatment:
#     -categorical vars

In [ ]:
# id -dont do it
# self indiactor -nochange
# match type- not req
# acct type -required 3 columns chosen for remaning crosstab then combine cols
    -
 #contributor type ARC is not ptesent in test so crosstab and then delete arc   
# ownership indicator no rpoblem
# account status not present in test Sold/Purchased ,Cancelled cross tab then delete                            
# credit limit CREDIT-LIMIT/SANC AMT  DISBURSED-AMT/HIGH CREDIT kbindiscretization
# installment not present in test data F04 F07  F08 crosstab then delete it
# asset class 2 not ptresent in test crosstab and remove it

In [42]:
train_b.ID

0              1
1              1
2              1
3              1
4              1
           ...  
560839    143393
560840    143393
560841    143393
560842    143394
560843    143395
Name: ID, Length: 560844, dtype: int64

In [49]:
train_SELF_INDICATOR=pd.crosstab(train_b.ID,train_b['SELF-INDICATOR'],dropna= True).reset_index()
test_SELF_INDICATOR=pd.crosstab(test_b.ID,test_b['SELF-INDICATOR'],dropna= True).reset_index()

In [50]:
train_MATCH_TYPE=pd.crosstab(train_b.ID,train_b['MATCH-TYPE'],dropna= True).reset_index()
test_MATCH_TYPE=pd.crosstab(test_b.ID,test_b['MATCH-TYPE'],dropna= True).reset_index()

In [54]:
x = set(train_bb['ACCT-TYPE'].unique())  
y = set(test_bb['ACCT-TYPE'].unique()) 
common = x.intersection(y)
print(list(common)) #now decide cols to keep 

['Mudra Loans   Shishu / Kishor / Tarun', 'Non-Funded Credit Facility', 'Business Loan Priority Sector  Agriculture', 'Gold Loan', 'Business Non-Funded Credit Facility-Priority Sector- Small Business', 'Personal Loan', 'Housing Loan', 'Individual', 'Business Loan Priority Sector  Small Business', 'Commercial Vehicle Loan', 'Microfinance Business Loan', 'Prime Minister Jaan Dhan Yojana - Overdraft', 'Microfinance Others', 'Overdraft', 'Business Non-Funded Credit Facility General', 'Loan to Professional', 'Two-Wheeler Loan', 'Business Loan Unsecured', 'Business Loan Against Bank Deposits', 'Auto Loan (Personal)', 'SHG Individual', 'Loan Against Shares / Securities', 'Construction Equipment Loan', 'Business Non-Funded Credit Facility-Priority Sector-Agriculture', 'Pradhan Mantri Awas Yojana - CLSS', 'Tractor Loan', 'Corporate Credit Card', 'Fleet Card', 'Consumer Loan', 'Business Loan Priority Sector  Others', 'Commercial Equipment Loan', 'Microfinance Personal Loan', 'Property Loan', 'Mi

In [81]:
train_ACCT_TYPE=pd.crosstab(train_b.ID,train_b['ACCT-TYPE'],dropna= True).reset_index()
test_ACCT_TYPE=pd.crosstab(test_b.ID,test_b['ACCT-TYPE'],dropna= True).reset_index()

cols_to_keep =['Tractor Loan','Gold Loan','Business Loan Priority Sector  Agriculture']

train_ACCT_TYPE["Others"] = pd.DataFrame(train_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
test_ACCT_TYPE["Others"] = pd.DataFrame(test_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
filter_cols =cols_to_keep + ['Others','ID'] 
train_ACCT_TYPE = train_ACCT_TYPE.filter(filter_cols)
test_ACCT_TYPE = test_ACCT_TYPE.filter(filter_cols)

In [56]:
train_CONTRIBUTOR_TYPE=pd.crosstab(train_b.ID,train_b['CONTRIBUTOR-TYPE'],dropna= True).drop(columns = ['ARC']).reset_index()
test_CONTRIBUTOR_TYPE=pd.crosstab(test_b.ID,test_b['CONTRIBUTOR-TYPE'],dropna= True).reset_index()


In [57]:
train_OWNERSHIP_IND=pd.crosstab(train_b.ID,train_b['OWNERSHIP-IND'],dropna= True).reset_index()
test_OWNERSHIP_IND=pd.crosstab(test_b.ID,test_b['OWNERSHIP-IND'],dropna= True).reset_index()


In [58]:
train_ACCOUNT_STATUS=pd.crosstab(train_b.ID,train_b['ACCOUNT-STATUS'],dropna= True).drop(columns=['Sold/Purchased','Cancelled']).reset_index()
test_ACCOUNT_STATUS=pd.crosstab(test_b.ID,test_b['ACCOUNT-STATUS'],dropna= True).reset_index()


In [59]:
train_INSTALLMENT_FREQUENCY=pd.crosstab(train_b.ID,train_b['INSTALLMENT-FREQUENCY'],dropna= True).drop(columns=['F04','F07','F08']).reset_index()
test_INSTALLMENT_FREQUENCY=pd.crosstab(test_b.ID,test_b['INSTALLMENT-FREQUENCY'],dropna= True).reset_index()

In [60]:
train_ASSET_CLASS=pd.crosstab(train_b.ID,train_b['ASSET_CLASS'],dropna= True).drop(columns=['2']).reset_index()
test_ASSET_CLASS=pd.crosstab(test_b.ID,test_b['ASSET_CLASS'],dropna= True).reset_index()

In [61]:
def onehotencoder(train,test,cols=[]):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    

    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def targetencoding(train,test,y_train,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc


def ordinalencoding(train,test,y_train,mapping,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'ordinal encoding for features {cat_features}')
    target_enc = ce.OrdinalEncoder(cols=cat_features,mapping = mapping)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_ordinal'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_ordinal'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [62]:
train_bb['DISBURSED-AMT/HIGH CREDIT'] = train_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')
test_bb['DISBURSED-AMT/HIGH CREDIT'] = test_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')

In [63]:
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].fillna(0)
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].fillna(0)
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].str.replace(',', '').astype('float')
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].str.replace(',', '').astype('float')

In [64]:

train_CURRENT = train_bb[['CURRENT-BAL','ID']].dropna()
test_CURRENT = test_bb[['CURRENT-BAL','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['CURRENT-BAL'].dropna().values.reshape(-1,1))

train_CURRENT['CURRENT-BAL'] = enc.transform(train_CURRENT['CURRENT-BAL'].values.reshape(-1,1))
test_CURRENT['CURRENT-BAL']= enc.transform(test_CURRENT['CURRENT-BAL'].values.reshape(-1,1))

In [65]:
train_CURRENT=pd.crosstab(train_CURRENT.ID,train_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT=pd.crosstab(test_CURRENT.ID,test_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT['3.0']=0

In [66]:

train_DISBURSED = train_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()
test_DISBURSED = test_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['DISBURSED-AMT/HIGH CREDIT'].dropna().values.reshape(-1,1))

train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'] = enc.transform(train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))
test_DISBURSED['DISBURSED-AMT/HIGH CREDIT']= enc.transform(test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))

In [67]:
train_DISBURSED=pd.crosstab(train_DISBURSED.ID,train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED=pd.crosstab(test_DISBURSED.ID,test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED['2.0']=0

In [68]:
#discretisation baes on clusters

train_tenure = train_bb[['TENURE','ID']].dropna()
test_tenure = test_bb[['TENURE','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['TENURE'].dropna().values.reshape(-1,1))

train_tenure['TENURE'] = enc.transform(train_tenure['TENURE'].values.reshape(-1,1))
test_tenure['TENURE']= enc.transform(test_tenure['TENURE'].values.reshape(-1,1))

In [69]:
train_tenure=pd.crosstab(train_tenure.ID,train_tenure['TENURE'],dropna= True).reset_index()
test_tenure=pd.crosstab(test_tenure.ID,test_tenure['TENURE'],dropna= True).reset_index()


In [82]:
train_SELF_INDICATOR
train_INSTALLMENT_FREQUENCY
train_ACCOUNT_STATUS
train_OWNERSHIP_IND
train_CONTRIBUTOR_TYPE
train_ACCT_TYPE

ACCT-TYPE,Tractor Loan,Gold Loan,Business Loan Priority Sector Agriculture,Others,ID
0,2,0,0,8,1
1,2,0,0,13,2
2,1,8,0,25,3
3,3,0,0,8,7
4,4,0,0,11,8
...,...,...,...,...,...
128650,1,0,0,143391,143390
128651,1,0,0,143391,143391
128652,3,0,0,143394,143393
128653,1,0,0,143394,143394


In [83]:
dataframes = [train_b.ID,train_SELF_INDICATOR,train_INSTALLMENT_FREQUENCY,train_ACCOUNT_STATUS,train_OWNERSHIP_IND,
              train_CONTRIBUTOR_TYPE,train_ACCT_TYPE,train_MATCH_TYPE,train_ASSET_CLASS,train_CURRENT,train_DISBURSED,
              train_tenure]
final_df_train_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [84]:
dataframes = [test_b.ID,test_SELF_INDICATOR,test_INSTALLMENT_FREQUENCY,test_ACCOUNT_STATUS,test_OWNERSHIP_IND,
              test_CONTRIBUTOR_TYPE,test_ACCT_TYPE,test_MATCH_TYPE,test_ASSET_CLASS,test_CURRENT,test_DISBURSED,test_tenure]
final_df_test_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [85]:
###### now train data 

In [86]:
train_dd = train_d.copy()
test_dd = test_d.copy()

In [87]:
train_dd.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [88]:
train_dd['SEX'] = train_dd['SEX'].fillna(train_dd['SEX'].mode()[0])
test_dd['SEX'] = test_dd['SEX'].fillna(test_dd['SEX'].mode()[0])

train_dd['AGE'] = train_dd['AGE'].fillna(train_dd['AGE'].mode()[0])
test_dd['AGE'] = test_dd['AGE'].fillna(test_dd['AGE'].mode()[0])

train_dd['MonthlyIncome'] = train_dd['MonthlyIncome'].fillna(train_dd['MonthlyIncome'].mode()[0])
test_dd['MonthlyIncome'] = test_dd['MonthlyIncome'].fillna(test_dd['MonthlyIncome'].mode()[0])

train_dd['ZiPCODE']  =train_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 
test_dd['ZiPCODE']  =test_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 

train_dd =train_dd.fillna(method = 'ffill') 
test_dd =test_dd.fillna(method = 'ffill') 

In [89]:
train_dd['AuthDate'] = pd.to_datetime(train_d['AuthDate'] ,errors='coerce')
test_dd['AuthDate'] = pd.to_datetime(test_d['AuthDate'] ,errors='coerce')

train_dd['DisbursalDate'] = pd.to_datetime(train_d['DisbursalDate'] ,errors='coerce')
test_dd['DisbursalDate'] = pd.to_datetime(test_d['DisbursalDate'] ,errors='coerce')

train_dd['MaturityDAte'] = pd.to_datetime(train_d['MaturityDAte'] ,errors='coerce')
test_dd['MaturityDAte'] = pd.to_datetime(test_d['MaturityDAte'] ,errors='coerce')

In [90]:
train_dd['AuthtoMatureDays'] = train_dd['AuthDate'] - train_dd['MaturityDAte']
test_dd['AuthtoMatureDays'] = test_dd['AuthDate'] - test_dd['MaturityDAte']

train_dd['AuthtoDisDays'] = train_dd['AuthDate'] - train_dd['DisbursalDate']
test_dd['AuthtoDisDays'] = test_dd['AuthDate'] - test_dd['DisbursalDate']

In [91]:
train_dd['AuthYear'] = train_dd['AuthDate'].dt.year 
test_dd['AuthtYear'] = test_dd['AuthDate'].dt.year 
train_dd['AuthMonth'] = train_dd['AuthDate'].dt.month 
test_dd['AuthtMonth'] = test_dd['AuthDate'].dt.month 


train_dd['MaturityYear'] = train_d['MaturityDAte'].dt.year 
test_dd['MaturityYear'] = test_d['MaturityDAte'].dt.year 

train_dd['MaturityMonth'] = train_d['MaturityDAte'].dt.month 
test_dd['MaturityMonth'] = test_d['MaturityDAte'].dt.month 

In [92]:
train_dd = train_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])
test_dd = test_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])

In [93]:
mapping_dict = {'No Top-up Service':0, '12-18 Months':1,'18-24 Months':2, '24-30 Months':3, 
                '30-36 Months':4,'36-48 Months':5,' > 48 Months':6}

In [94]:
train_dd['Top-up Month'] = train_dd['Top-up Month'].replace(mapping_dict) 

In [95]:
mapping_dict_f= [
{'col': 'Frequency', 'mapping': {'BI-Monthly':0, 'Monthly': 1, 'Quatrly': 2,  'Half Yearly': 3}}
]

In [96]:
f1,f2 = ordinalencoding(train_dd,test_dd,train_dd['Top-up Month'],drop =True,cols=['Frequency'],mapping= mapping_dict_f)

ModuleNotFoundError: ignored

In [34]:
f11,f22 = onehotencoder(f1,f2,cols=['InstlmentMode','LoanStatus','SEX'])

NameError: ignored

In [35]:
f111,f222 = targetencoding(f11,f22,f11['Top-up Month'],drop=True,cols=['PaymentMode','State','BranchID','ZiPCODE','ManufacturerID']) #because branch id and area same

NameError: ignored

In [ ]:
dataframes = [f111,final_df_train_beauro]
final_df_train = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [36]:
dataframes = [f222,final_df_test_beauro]
final_df_test = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

NameError: ignored

In [ ]:
final_df_train = final_df_train.fillna(0)
final_df_test = final_df_test.fillna(0)

In [ ]:
#Null value treatment

In [ ]:
final_df_train.columns

In [ ]:
cols = []

In [ ]:
exp_clf102 = setup(data = final_df_train, target = 'Top-up Month', session_id=123,
                  normalize = True, 
                  transformation = True, 
                  ignore_low_variance = True,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95,
                 feature_interaction=True,
                   categorical_features=cols
                  )


In [ ]:
rf = create_model('rf')

In [ ]:
tune_rf = create_model('rf')

In [ ]:
et = create_model('xgboost')

In [ ]:
tune_et = tune_model('xgboost')

In [ ]:
plot_model(estimator=tune_et, plot='feature')

In [ ]:
plot_model(tune_et, plot = 'confusion_matrix')

In [ ]:
blend_hard = blend_models()

In [ ]:
blend_soft = blend_models(method = 'soft')

In [ ]:
blend_specific_hard = blend_models(estimator_list = [tune_et,tune_ridge], method = 'hard')